In [1]:
import os
import matplotlib.pyplot as plt
from scipy.io import wavfile
import noisereduce
import numpy as np
import pandas as pd
import librosa
import skimage.io
from pathlib import Path

SPECTROGRAM_RESOLUTION = 128

AUDIO = "..\\tensorflow-speech-recognition-challenge\\train\\audio"

SILENCE_DIR = "_background_noise_"

SPECTROGRAMS_DIR = "spectrograms_v2_reduced_noise_further"

TARGET_DURATION = 1.0

def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

def spectrogram_image(y, sr, out, hop_length, n_mels):
    # use log-melspectrogram
    mels = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels,
                                            n_fft=hop_length*2, hop_length=hop_length)
    mels = np.log(mels + 1e-9) # add small number to avoid log(0)

    # min-max scale to fit inside 8-bit range
    img = scale_minmax(mels, 0, 255).astype(np.uint8)
    img = np.flip(img, axis=0) # put low frequencies at the bottom in image
    img = 255-img # invert. make black==more energy

    # save as PNG
    skimage.io.imsave(out, img)

def find_longest_audio(root_dir) :
    longest_duration = 0

    for dirpath, dirnames, filenames in os.walk(root_dir):
        if SILENCE_DIR in dirpath:
            continue

        for file in filenames:
            if  file.endswith(".wav"):
                file_path =  os.path.join(dirpath, file)
                try:
                    y, sr = librosa.load(file_path, sr=None)
                    duration = librosa.get_duration(y=y,sr=sr)
                    print(duration)
                    if duration > longest_duration:
                        longest_duration = duration
                except Exception as ex:
                    print(f"Error processing {file}: {ex}")
    return longest_duration

def pad_spectrogram(spec, target_length):
    if spec.shape[1] < target_length:
        # Pad with zeros on the right
        padding = target_length - spec.shape[1]
        spec = np.pad(spec, ((0, 0), (0, padding)), mode='constant')
    return spec

#https://stackoverflow.com/questions/56719138/how-can-i-save-a-librosa-spectrogram-plot-as-a-specific-sized-image/57204349#57204349


# for directory in directories:
#     for filename in os.listdir(directory):
#         if filename.startswith('.'):
#             continue
#         if filename.endswith('.wav'):
#             filepath = os.path.join(directory, filename)
            
#             try:
#                 waveform, sample_rate = librosa.load(filepath)

#                 target_samples = int(TARGET_DURATION * sample_rate)
#                 waveform = librosa.util.fix_length(waveform, target_samples)

#                 print(len(waveform)/sample_rate)
                
#                 waveform_r = noisereduce.reduce_noise(y=waveform,sr=sample_rate) # reducing noise
#                 # waveform_tr, index = librosa.effects.trim(waveform_r,top_db=20) #trimming silence on both ends
                
#                 base_filename = os.path.splitext(filename)[0]
                        
#                 spectrogram_image(no_silence,sample_rate,f"../spectrograms_v2/{base_filename}.png",SPECTROGRAM_RESOLUTION,SPECTROGRAM_RESOLUTION)

#             except ValueError as e:
#                 print(f"Error reading {filename}: {e}")
#             except Exception as e:
#                 print(f"An unexpected error occurred with {filename}: {e}")

D:\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
D:\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
D:\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [10]:
target_dirs = {"yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go"}

# Get the list of all directories
all_dirs = [d for d in os.listdir(AUDIO) if os.path.isdir(os.path.join(AUDIO, d))]

# Include only the excluded directories
other_dirs = [d for d in all_dirs if d not in target_dirs]

for dirpath, dirnames, filenames in os.walk(AUDIO):
    if SILENCE_DIR in dirpath:
        continue

    for file in filenames:
        if  file.endswith(".wav"):
            file_path =  os.path.join(dirpath, file)
            try:
                waveform, sample_rate = librosa.load(file_path)

                dir_name = Path(file_path).parent.name

                target_samples = int(TARGET_DURATION * sample_rate)
                waveform = librosa.util.fix_length(data = waveform, size = target_samples)
                
                waveform = noisereduce.reduce_noise(y=waveform,sr=sample_rate) # reducing noise   
                waveform = noisereduce.reduce_noise(y=waveform,sr=sample_rate, thresh_n_mult_nonstationary = 12, n_fft = 512) # reducing noise                   
                base_filename = os.path.splitext(file)[0]

                target_dir = f"../{SPECTROGRAMS_DIR}/{dir_name}"
                os.makedirs(target_dir, exist_ok=True)
                target_file_name = f"{base_filename}_{dir_name}_.png"
                target_path = os.path.join(target_dir, target_file_name)
                        
                spectrogram_image(waveform,sample_rate,target_path,512,128)

            except ValueError as e:
                print(f"Error reading {file}: {e}")
            except Exception as e:
                print(f"An unexpected error occurred with {file}: {e}")


D:\anaconda3\Lib\site-packages\noisereduce\spectralgate\nonstationary.py:70: RuntimeWarning: invalid value encountered in divide
  sig_mult_above_thresh = (abs_sig_stft - sig_stft_smooth) / sig_stft_smooth


An unexpected error occurred with 3e7124ba_nohash_0.wav: Audio buffer is not finite everywhere


C:\Users\macie\AppData\Local\Temp\ipykernel_8160\4031475547.py:38: UserWarning: ../spectrograms_v2_reduced_noise_further/zero\439c84f4_nohash_0_zero_.png is a low contrast image
  skimage.io.imsave(out, img)


In [11]:
import random
import shutil
OTHERS = 2375
ROOT_DIR = f"../{SPECTROGRAMS_DIR}"

exclude_dirs = {"yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "other"}

# Get the list of all directories
all_dirs = [d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))]

# Include only the excluded directories
other_dirs = [d for d in all_dirs if d not in exclude_dirs]

other_dir = os.path.join(ROOT_DIR, "other")

os.makedirs(other_dir, exist_ok=True)

samples_per_dir = OTHERS // len(other_dirs)

for dir in other_dirs:
    dir_path = os.path.join(ROOT_DIR, dir)
    files = os.listdir(dir_path)

    selected_files = random.sample(files, samples_per_dir)

    for file_namne in selected_files:
        source_path = os.path.join(dir_path, file_namne)
        target_path = os.path.join(other_dir, file_namne)

        shutil.copy(source_path, target_path)

In [12]:
import os
import matplotlib.pyplot as plt
from scipy.io import wavfile
import noisereduce
import numpy as np
import pandas as pd
import librosa
import skimage.io

SILENCE_DIR = "..\\tensorflow-speech-recognition-challenge\\train\\audio\\_background_noise_"

def slice_silence(waveform, sample_rate, segment_length,filename):

    segment_length_frames = int(segment_length * sample_rate)
    num_segments = len(waveform) // segment_length_frames

    for i in range(num_segments):
        start_frame = i * segment_length_frames
        end_frame = start_frame + segment_length_frames
        slice_waveform = waveform[start_frame:end_frame]

        spectrogram_image(slice_waveform,sample_rate,f"{filename}_{i}.png",512,128)


for i in range (6):
    for file_name in os.listdir(SILENCE_DIR):
        if file_name.endswith(".wav"):
            file_path =  os.path.join(SILENCE_DIR, file_name)
            try:
                waveform, sample_rate = librosa.load(file_path)
                
                base_filename = os.path.splitext(file_name)[0]

                dir_name = Path(file_path).parent.name
    
                target_dir = f"../{SPECTROGRAMS_DIR}/silence"
                os.makedirs(target_dir, exist_ok=True)
                target_file_name = f"{base_filename}_{file_name}_{i}"
    
                target = os.path.join(target_dir, target_file_name)
    
                slice_silence(waveform, sample_rate, 1, target)
                        
            except ValueError as e:
                print(f"Error reading {file_name}: {e}")
            except Exception as e:
                print(f"An unexpected error occurred with {file_name}: {e}")

In [13]:
image_folders = exclude_dirs = {"yes", "no", "up", "down", "left", "right", "on", "off", "stop", "go", "silence", "other"}

all_dirs = os.listdir(ROOT_DIR)

image_folders = [d for d in all_dirs if d in image_folders ]

image_folder = "..//image_folder_reduced_noise_further"

os.makedirs(image_folder, exist_ok=True)

for dir in image_folders:
    source = os.path.join(ROOT_DIR, dir)
    target = os.path.join(image_folder, dir)
    shutil.move(source, target)